In [67]:
import pandas as pd
import json
from pathlib import Path

pd.set_option("display.max_rows", None)

In [68]:
associazione_filepath = Path("./data/abgrid/vega/vega_ii/associazione.xlsx")
vega_ii_json_filepath = Path("./data/abgrid/vega/vega_ii/vega_ii_lud_data.json")
abgrid_long = pd.read_excel("./analysis/abgrid_long.xlsx").set_index("id").sort_index()
abgrid_wide = pd.read_excel("./analysis/abgrid_wide.xlsx").set_index("id").sort_index()
safa_anx =  pd.read_excel("./analysis/safa_anx.xlsx").set_index("id").sort_index()
mmpia = pd.read_excel("./analysis/mmpia.xlsx").set_index("id").sort_index()
accme = pd.read_excel("./analysis/accme.xlsx").set_index("id").sort_index()
associazione_df = pd.read_excel(associazione_filepath)

with open(vega_ii_json_filepath, "r") as fin:
    vega_ii_json = json.loads(fin.read())

In [82]:
df_a = pd.DataFrame.from_dict(vega_ii_json["vega_ii_lud_g1"]["relevant_nodes_ab"]["a"])
df_a.insert(0, "group", "A")
df_b = pd.DataFrame.from_dict(vega_ii_json["vega_ii_lud_g1"]["relevant_nodes_ab"]["b"])
df_b.insert(0, "group", "B")
df = pd.concat([df_a, df_b])
df = df[df["metrics"].str.len() > 1]
df = df.loc[:, ["id", "group"]].set_index("id")
final = associazione_df.set_index("lettera").reindex(df.index)
final["group"] = df["group"]
final["soggetto"] = final["soggetto"].str.lower() 
final = (
    final
        .merge(mmpia, left_on="soggetto", right_index=True)
        .groupby("group")[mmpia.filter(regex="_scale_t").columns]
        .mean().T
        .astype(int)
)
final[final.A.sub(final.B).abs().ge(5)]

group,A,B
Hy_scale_t,53,48
A-anx_scale_t,38,33
A-dep_scale_t,39,34
A-sod_scale_t,43,48
PRO_scale_t,39,34
R_scale_t,51,58
Si1_scale_t,36,41
Si2_scale_t,48,57
Pd2_scale_t,41,47
Pa3_scale_t,69,74
